In [1]:
# -------------------------
# Configuración base
# -------------------------
import os
import warnings, json, time, random, gc
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

BASE_DIR = r"C:\Users\juanf\Downloads\EAFIT\Semestre II\PI-2\PI_2"

print("BASE_DIR:", BASE_DIR)

BASE_DIR: C:\Users\juanf\Downloads\EAFIT\Semestre II\PI-2\PI_2


In [2]:
# ==========================
#  Crear máscaras temporales en df_feats
# ==========================

df_feats = pd.read_csv(r"C:\Users\juanf\Downloads\EAFIT\Semestre II\PI-2\PI_2\data\processed\train_features.csv", parse_dates=["date"])


splits = {
    "fold1": {
        "train_start": datetime(2013, 1, 1),
        "train_end"  : datetime(2016, 12, 31),
        "valid_start": datetime(2017, 1, 1),
        "valid_end"  : datetime(2017, 3, 31),
    },
    "fold2": {
        "train_start": datetime(2013, 1, 1),
        "train_end"  : datetime(2017, 3, 31),
        "valid_start": datetime(2017, 4, 1),
        "valid_end"  : datetime(2017, 6, 30),
    },
    "fold3": {
        "train_start": datetime(2013, 1, 1),
        "train_end"  : datetime(2017, 6, 30),
        "valid_start": datetime(2017, 7, 1),
        "valid_end"  : datetime(2017, 9, 30),
    },
    "test_interno": {
        "start": datetime(2017, 10, 1),
        "end"  : datetime(2017, 12, 31),
    }
}

def create_fold_mask(df, start, end):
    return (df["date"] >= start) & (df["date"] <= end)

fold_masks = {
    fold_name: {
        "train_mask": create_fold_mask(df_feats, f["train_start"], f["train_end"]),
        "valid_mask": create_fold_mask(df_feats, f["valid_start"], f["valid_end"])
    }
    for fold_name, f in splits.items()
    if fold_name != "test_interno"
}

fold_masks

{'fold1': {'train_mask': 0          True
  1          True
  2          True
  3          True
  4          True
            ...  
  897995    False
  897996    False
  897997    False
  897998    False
  897999    False
  Name: date, Length: 898000, dtype: bool,
  'valid_mask': 0         False
  1         False
  2         False
  3         False
  4         False
            ...  
  897995    False
  897996    False
  897997    False
  897998    False
  897999    False
  Name: date, Length: 898000, dtype: bool},
 'fold2': {'train_mask': 0          True
  1          True
  2          True
  3          True
  4          True
            ...  
  897995    False
  897996    False
  897997    False
  897998    False
  897999    False
  Name: date, Length: 898000, dtype: bool,
  'valid_mask': 0         False
  1         False
  2         False
  3         False
  4         False
            ...  
  897995    False
  897996    False
  897997    False
  897998    False
  897999    False
  Na

Para implementar un esquema riguroso de validación temporal, se generaron máscaras booleanas (train_mask y valid_mask) que identifican, para cada observación del dataset, si pertenece al conjunto de entrenamiento o al conjunto de validación dentro de cada fold. Estas máscaras permiten segmentar el dataset sin romper la estructura temporal, evitando fugas de información y garantizando que el modelo aprende únicamente de datos pasados.  

Las máscaras también facilitan la reproducibilidad y eficiencia computacional, ya que permiten seleccionar de manera precisa y vectorizada las observaciones de cada ventana temporal. Este enfoque es consistente con las mejores prácticas de Series Temporales en la industria y asegura que el esquema de validación refleja fielmente la dinámica real de forecasting en entornos productivos.

In [3]:
# ==========================
#  Definir X e y
# ==========================

feature_cols = [
    "store", "item", "day", "dayofweek", "weekofyear", "month", "year", "is_weekend",
    "lag_1", "lag_7", "lag_14", "lag_28",
    "rolling_mean_7", "rolling_mean_30",
    "rolling_std_7", "rolling_std_30"
]

X = df_feats[feature_cols]
y = df_feats["sales"]

X.head()

,store,item,day,dayofweek,weekofyear,month,year,is_weekend,lag_1,lag_7,lag_14,lag_28,rolling_mean_7,rolling_mean_30,rolling_std_7,rolling_std_30
0,1,1,31,3,5,1,2013,0,9.0,8.0,16.0,14.0,10.285714,10.500000,2.751623,3.104502
1,1,1,1,4,5,2,2013,0,13.0,14.0,7.0,13.0,11.000000,10.500000,2.708013,3.104502
2,1,1,2,5,5,2,2013,1,11.0,12.0,18.0,10.0,10.571429,10.500000,2.370453,3.104502
3,1,1,3,6,5,2,2013,1,21.0,12.0,15.0,12.0,11.857143,10.733333,4.634241,3.600128
4,1,1,4,0,6,2,2013,0,15.0,11.0,8.0,10.0,12.285714,10.800000,4.785892,3.661543


Aquí construimos la matriz de características **X** y el vector objetivo **y**.  

La definición de **X** e **y** constituye la formalización del problema de predicción de ventas como una tarea supervisada de regresión. El vector objetivo **y** contiene las ventas históricas, mientras que la matriz **X** integra todos los atributos diseñados para capturar estacionalidad, autocorrelación, tendencia y variabilidad de la serie temporal. Este conjunto de características permite que un modelo global como Random Forest aprenda patrones compartidos entre múltiples tiendas y productos, maximizando la eficiencia y la capacidad predictiva del enfoque basado en machine learning.

In [4]:
# ==========================
#  Entrenar Random Forest global por fold
# ==========================

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

rf_results = []

# Modelo base 
rf_model_base = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42
)

for fold_name, masks in fold_masks.items():

    print(f"\nEntrenando fold: {fold_name}")

    X_train = X[masks["train_mask"]]
    y_train = y[masks["train_mask"]]

    X_valid = X[masks["valid_mask"]]
    y_valid = y[masks["valid_mask"]]

    print(f"  Tamaño train: {X_train.shape}, Tamaño valid: {X_valid.shape}")

    # Entrenar
    rf_model_base.fit(X_train, y_train)

    # Predecir
    preds = rf_model_base.predict(X_valid)

    # Métricas
    rmse = mean_squared_error(y_valid, preds)
    mae  = mean_absolute_error(y_valid, preds)
    rmse = np.sqrt(rmse)

    # MAPE (evita división entre cero)
    mape = (np.abs((y_valid - preds) / y_valid.replace(0, np.nan))).mean() * 100

    rf_results.append({
        "fold": fold_name,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape
    })

# Convertir resultados en dataframe
df_rf_results = pd.DataFrame(rf_results)
df_rf_results


Entrenando fold: fold1
  Tamaño train: (715500, 16), Tamaño valid: (45000, 16)

Entrenando fold: fold2
  Tamaño train: (760500, 16), Tamaño valid: (45500, 16)

Entrenando fold: fold3
  Tamaño train: (806000, 16), Tamaño valid: (46000, 16)


,fold,RMSE,MAE,MAPE
0,fold1,7.189267,5.538061,14.782941
1,fold2,8.582785,6.626137,11.692515
2,fold3,8.686457,6.706798,11.667471


El entrenamiento por folds temporales demostró que el modelo Random Forest Global presenta un desempeño sólido y estable a lo largo de distintos períodos de validación. Los tamaños crecientes del conjunto de entrenamiento permiten que el modelo incorpore progresivamente más estacionalidades y patrones históricos, lo que se refleja en una mejora del MAPE en los dos últimos folds. Estos resultados muestran que el enfoque global no solo es más eficiente que los modelos locales tipo SARIMA, sino que además logra capturar patrones comunes entre tiendas y productos, beneficiándose de la naturaleza multiserie del dataset.

In [5]:
# ==========================
#  Predicciones por fold (Random Forest)
# ==========================

rf_predictions = []

for fold_name, masks in fold_masks.items():

    print(f"\nObteniendo predicciones para fold: {fold_name}")

    X_train = X[masks["train_mask"]]
    y_train = y[masks["train_mask"]]
    X_valid = X[masks["valid_mask"]]
    y_valid = y[masks["valid_mask"]]

    # Entrenar
    rf_model_base.fit(X_train, y_train)

    # Predecir
    preds = rf_model_base.predict(X_valid)

    # Guardar resultados con índices y store-item
    fold_df = df_feats[masks["valid_mask"]][["store","item","date","sales"]].copy()
    fold_df["prediction"] = preds
    fold_df["fold"] = fold_name

    rf_predictions.append(fold_df)

# Unir todos los folds
df_rf_all_preds = pd.concat(rf_predictions).reset_index(drop=True)
df_rf_all_preds.head()


Obteniendo predicciones para fold: fold1

Obteniendo predicciones para fold: fold2

Obteniendo predicciones para fold: fold3


,store,item,date,sales,prediction,fold
0,1,1,2017-01-01,19,20.317540,fold1
1,1,1,2017-01-02,15,13.533866,fold1
2,1,1,2017-01-03,10,15.396713,fold1
3,1,1,2017-01-04,16,15.722935,fold1
4,1,1,2017-01-05,14,16.064162,fold1


In [6]:
# ==========================
#  Métricas por tienda-item (RF)
# ==========================

def compute_metrics(group):
    rmse = np.sqrt(mean_squared_error(group["sales"], group["prediction"]))
    mae = mean_absolute_error(group["sales"], group["prediction"])
    mape = (np.abs((group["sales"] - group["prediction"]) / group["sales"].replace(0, np.nan))).mean() * 100
    return pd.Series({"RMSE": rmse, "MAE": mae, "MAPE": mape})

df_rf_metrics = df_rf_all_preds.groupby(["store","item","fold"]).apply(compute_metrics).reset_index()

df_rf_metrics.head()

,store,item,fold,RMSE,MAE,MAPE
0,1,1,fold1,4.313039,3.538330,24.350854
1,1,1,fold2,5.478640,4.176541,16.948847
2,1,1,fold3,4.709093,3.890641,16.332070
3,1,2,fold1,6.936185,5.569984,13.692650
4,1,2,fold2,10.234321,7.916197,11.926550


El análisis por tienda–producto confirma que el modelo Random Forest Global es capaz de capturar patrones comunes compartidos entre múltiples series, logrando un desempeño estable y razonablemente preciso en la mayoría de los casos. Sin embargo, se observan diferencias naturales en el nivel de error entre series individuales, derivadas de la estacionalidad, volatilidad y escala de cada producto y tienda. Esto demuestra la importancia de evaluar modelos globales no solo de forma agregada, sino también a nivel desagregado, para identificar patrones particulares y oportunidades de refinamiento.

In [7]:
# ==========================
# Guardado de resultados
# ==========================
output_path = Path(BASE_DIR) / "data" / "processed" / "random_forest_global_test.csv"
df_rf_metrics.to_csv(output_path, index=False)